In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

## 1. Define Translation Dictionaries

In [17]:
# Column name translations
COLUMN_TRANSLATION = {
    '地点名称': 'location_name',
    '地点别名': 'location_alias',
    '省份名称': 'province',
    '城市代码': 'city_code',
    '城市名称': 'city',
    '区域代码': 'district_code',
    '区域名称': 'district',
    '商圈': 'business_district',
    '详细地址': 'address',
    '经度': 'longitude',
    '纬度': 'latitude',
    '标签': 'tags',
    '邮政编码': 'postcode',
    '大类代码': 'category_code',
    '大类名称': 'category_name',
    '类型代码': 'type_code',
    '所有类型': 'all_types',
    '商业类型': 'business_type',
    '评分': 'rating',
    '消费（元）': 'price_yuan'
}

# Province translations
PROVINCE_TRANSLATION = {
    '北京': 'Beijing',
    '上海': 'Shanghai',
    '天津': 'Tianjin',
    '重庆': 'Chongqing',
    '广东': 'Guangdong',
    '江苏': 'Jiangsu',
    '浙江': 'Zhejiang',
    '山东': 'Shandong',
    '四川': 'Sichuan',
    '河南': 'Henan',
    '湖北': 'Hubei',
    '湖南': 'Hunan',
    '福建': 'Fujian',
    '安徽': 'Anhui',
    '河北': 'Hebei',
    '陕西': 'Shaanxi',
    '辽宁': 'Liaoning',
    '江西': 'Jiangxi',
    '云南': 'Yunnan',
    '广西': 'Guangxi',
    '山西': 'Shanxi',
    '吉林': 'Jilin',
    '黑龙江': 'Heilongjiang',
    '贵州': 'Guizhou',
    '甘肃': 'Gansu',
    '内蒙古': 'Inner Mongolia',
    '新疆': 'Xinjiang',
    '海南': 'Hainan',
    '宁夏': 'Ningxia',
    '青海': 'Qinghai',
    '西藏': 'Tibet',
    '香港': 'Hong Kong',
    '澳门': 'Macau',
    '台湾': 'Taiwan'
}

print("✅ Translation dictionaries defined")
print(f"   • {len(COLUMN_TRANSLATION)} column mappings")
print(f"   • {len(PROVINCE_TRANSLATION)} province translations")

✅ Translation dictionaries defined
   • 20 column mappings
   • 34 province translations


## 2. Load Chinese EV Station Data

In [18]:
print("📂 Loading Chinese EV station data...")
df_chinese = pd.read_excel('../data/raw/CnOpenData全国充电站分布数据（样本数据）.xlsx', engine='openpyxl')

print(f"✓ Loaded {len(df_chinese):,} records")
print(f"\n📋 Original columns:")
for i, col in enumerate(df_chinese.columns, 1):
    print(f"   {i}. {col}")

print(f"\n📊 Sample data:")
df_chinese.head(3)

📂 Loading Chinese EV station data...
✓ Loaded 2,002 records

📋 Original columns:
   1. 地点名称
   2. 地点别名
   3. 省份名称
   4. 城市代码
   5. 城市名称
   6. 区域代码
   7. 区域名称
   8. 商圈
   9. 详细地址
   10. 经度
   11. 纬度
   12. 标签
   13. 邮政编码
   14. 大类代码
   15. 大类名称
   16. 类型代码
   17. 所有类型
   18. 商业类型
   19. 评分
   20. 消费（元）

📊 Sample data:
✓ Loaded 2,002 records

📋 Original columns:
   1. 地点名称
   2. 地点别名
   3. 省份名称
   4. 城市代码
   5. 城市名称
   6. 区域代码
   7. 区域名称
   8. 商圈
   9. 详细地址
   10. 经度
   11. 纬度
   12. 标签
   13. 邮政编码
   14. 大类代码
   15. 大类名称
   16. 类型代码
   17. 所有类型
   18. 商业类型
   19. 评分
   20. 消费（元）

📊 Sample data:


,地点名称,地点别名,省份名称,城市代码,城市名称,区域代码,区域名称,商圈,详细地址,经度,纬度,标签,邮政编码,大类代码,大类名称,类型代码,所有类型,商业类型,评分,消费（元）
0,比亚迪电动汽车充电站,NaN,北京市,10.0,北京市,110112.0,通州区,NaN,科创东五街1号,116.551189,39.813862,NaN,NaN,1.0,汽车服务,11100.0,"{汽车服务,充电站,充电站}",NaN,NaN,NaN
1,汽车充电站(广阳西路),NaN,北京市,10.0,北京市,110111.0,房山区,NaN,广阳西路2号(成教中心门口),116.156761,39.757437,NaN,NaN,1.0,汽车服务,11100.0,"{汽车服务,充电站,充电站}",NaN,NaN,NaN
2,国家电网汽车充电站(晋元庄路),NaN,北京市,10.0,北京市,110107.0,石景山区,NaN,晋元庄路附近,116.211713,39.927628,NaN,NaN,1.0,汽车服务,11100.0,"{汽车服务,充电站,充电站}",NaN,NaN,NaN


## 3. Translate Columns and Province Names

In [19]:
print("🔤 Translating column names...")
df_chinese.columns = [COLUMN_TRANSLATION.get(col, col) for col in df_chinese.columns]
print(f"✓ Translated columns: {list(df_chinese.columns)}")

print("\n🌏 Translating province names...")
if 'province' in df_chinese.columns:
    df_chinese['province'] = df_chinese['province'].map(
        lambda x: PROVINCE_TRANSLATION.get(x, x) if pd.notna(x) else x
    )
    translated_count = df_chinese['province'].isin(PROVINCE_TRANSLATION.values()).sum()
    print(f"✓ Translated {translated_count:,} province names")

if 'city' in df_chinese.columns:
    df_chinese['city'] = df_chinese['city'].map(
        lambda x: PROVINCE_TRANSLATION.get(x, x) if pd.notna(x) else x
    )

print("\n📊 Province distribution:")
print(df_chinese['province'].value_counts().head(10))

🔤 Translating column names...
✓ Translated columns: ['location_name', 'location_alias', 'province', 'city_code', 'city', 'district_code', 'district', 'business_district', 'address', 'longitude', 'latitude', 'tags', 'postcode', 'category_code', 'category_name', 'type_code', 'all_types', 'business_type', 'rating', 'price_yuan']

🌏 Translating province names...
✓ Translated 0 province names

📊 Province distribution:
province
台湾省        871
香港特别行政区    189
北京市        163
广东省         93
江苏省         86
上海市         79
浙江省         77
山东省         70
河南省         42
安徽省         33
Name: count, dtype: int64


## 4. Map to Target Schema (ev_stations_2025.csv format)

In [20]:
print("🔄 Mapping to target schema...")

# Generate unique IDs starting from 500000 to avoid conflicts
start_id = 500000
df_chinese_mapped = pd.DataFrame()

# Map columns according to ev_stations_2025.csv schema
df_chinese_mapped['id'] = range(start_id, start_id + len(df_chinese))
df_chinese_mapped['title'] = df_chinese['location_name'].fillna('Unknown Station')
df_chinese_mapped['address'] = df_chinese['address'].fillna('')
df_chinese_mapped['town'] = df_chinese['district'].fillna('')
df_chinese_mapped['state'] = df_chinese['province'].fillna('')
df_chinese_mapped['postcode'] = df_chinese['postcode'].fillna('')
df_chinese_mapped['country'] = 'CN'
df_chinese_mapped['lat'] = df_chinese['latitude']
df_chinese_mapped['lon'] = df_chinese['longitude']
df_chinese_mapped['operator'] = df_chinese['business_type'].fillna('(Unknown Operator)')
df_chinese_mapped['status'] = 'Operational'
df_chinese_mapped['num_connectors'] = 1  # Conservative estimate
df_chinese_mapped['connector_types'] = 'Type 2 (Socket Only)'  # Common in China
df_chinese_mapped['date_added'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S+00:00')

print(f"✓ Mapped {len(df_chinese_mapped):,} records to target schema")
print(f"\n📋 Mapped columns: {list(df_chinese_mapped.columns)}")
print(f"\n📊 Sample mapped data:")
df_chinese_mapped.head(3)

🔄 Mapping to target schema...
✓ Mapped 2,002 records to target schema

📋 Mapped columns: ['id', 'title', 'address', 'town', 'state', 'postcode', 'country', 'lat', 'lon', 'operator', 'status', 'num_connectors', 'connector_types', 'date_added']

📊 Sample mapped data:


,id,title,address,town,state,postcode,country,lat,lon,operator,status,num_connectors,connector_types,date_added
0,500000,比亚迪电动汽车充电站,科创东五街1号,通州区,北京市,,CN,39.813862,116.551189,(Unknown Operator),Operational,1,Type 2 (Socket Only),2025-11-14 00:51:05+00:00
1,500001,汽车充电站(广阳西路),广阳西路2号(成教中心门口),房山区,北京市,,CN,39.757437,116.156761,(Unknown Operator),Operational,1,Type 2 (Socket Only),2025-11-14 00:51:05+00:00
2,500002,国家电网汽车充电站(晋元庄路),晋元庄路附近,石景山区,北京市,,CN,39.927628,116.211713,(Unknown Operator),Operational,1,Type 2 (Socket Only),2025-11-14 00:51:05+00:00


## 5. Load International EV Station Data

In [21]:
print("📂 Loading international EV station data...")
df_international = pd.read_csv('../data/raw/ev_stations_2025.csv')

print(f"✓ Loaded {len(df_international):,} records")
print(f"✓ Columns: {list(df_international.columns)}")
print(f"✓ Countries: {df_international['country'].nunique()}")

print(f"\n🌍 Top countries in international dataset:")
print(df_international['country'].value_counts().head(10))

📂 Loading international EV station data...
✓ Loaded 10,000 records
✓ Columns: ['id', 'title', 'address', 'town', 'state', 'postcode', 'country', 'lat', 'lon', 'operator', 'status', 'num_connectors', 'connector_types', 'date_added']
✓ Countries: 72

🌍 Top countries in international dataset:
country
CA    4135
US    2266
ES     888
RU     392
MY     377
TR     377
DK     296
BR     199
IT     134
FR     112
Name: count, dtype: int64


## 6. Merge Datasets

In [22]:
print("🔗 Merging datasets...")

# Ensure same columns in both datasets
cols = df_international.columns.tolist()
df_chinese_aligned = df_chinese_mapped[cols]

# Concatenate
df_merged = pd.concat([df_international, df_chinese_aligned], ignore_index=True)

print(f"✓ Merged dataset: {len(df_merged):,} total records")
print(f"  • International records: {len(df_international):,}")
print(f"  • Chinese records: {len(df_chinese_aligned):,}")
print(f"  • Total countries: {df_merged['country'].nunique()}")

print(f"\n🌍 Top 10 countries by station count:")
country_counts = df_merged['country'].value_counts().head(10)
for country, count in country_counts.items():
    pct = count / len(df_merged) * 100
    print(f"   {country:>3}: {count:>7,} stations ({pct:>5.1f}%)")

🔗 Merging datasets...
✓ Merged dataset: 12,002 total records
  • International records: 10,000
  • Chinese records: 2,002
  • Total countries: 72

🌍 Top 10 countries by station count:
    CA:   4,135 stations ( 34.5%)
    US:   2,266 stations ( 18.9%)
    CN:   2,003 stations ( 16.7%)
    ES:     888 stations (  7.4%)
    RU:     392 stations (  3.3%)
    TR:     377 stations (  3.1%)
    MY:     377 stations (  3.1%)
    DK:     296 stations (  2.5%)
    BR:     199 stations (  1.7%)
    IT:     134 stations (  1.1%)


## 7. Data Quality Checks

In [23]:
print("🔍 Data Quality Checks:")

# Check for missing coordinates
missing_coords = df_merged[df_merged['lat'].isna() | df_merged['lon'].isna()]
print(f"\n1. Missing coordinates: {len(missing_coords):,} records")

# Check for duplicate IDs
duplicate_ids = df_merged[df_merged.duplicated(subset=['id'], keep=False)]
print(f"2. Duplicate IDs: {len(duplicate_ids):,} records")

# Check coordinate ranges (valid lat: -90 to 90, lon: -180 to 180)
invalid_coords = df_merged[
    (df_merged['lat'] < -90) | (df_merged['lat'] > 90) |
    (df_merged['lon'] < -180) | (df_merged['lon'] > 180)
]
print(f"3. Invalid coordinates: {len(invalid_coords):,} records")

# Status distribution
print(f"\n⚡ Status distribution:")
status_counts = df_merged['status'].value_counts()
for status, count in status_counts.items():
    pct = count / len(df_merged) * 100
    print(f"   {status:30}: {count:>7,} ({pct:>5.1f}%)")

# Operator distribution (top 10)
print(f"\n🏢 Top 10 operators:")
operator_counts = df_merged['operator'].value_counts().head(10)
for operator, count in operator_counts.items():
    print(f"   {operator[:40]:40}: {count:>7,}")

print("\n✅ All quality checks complete!")

🔍 Data Quality Checks:

1. Missing coordinates: 2 records
2. Duplicate IDs: 0 records
3. Invalid coordinates: 0 records

⚡ Status distribution:
   Operational                   :  11,303 ( 94.2%)
   Planned For Future Date       :     302 (  2.5%)
   Temporarily Unavailable       :     176 (  1.5%)
   Not Operational               :     136 (  1.1%)
   Unknown                       :      80 (  0.7%)
   Partly Operational (Mixed)    :       5 (  0.0%)

🏢 Top 10 operators:
   (Unknown Operator)                      :   2,425
   Circuit Electrique                      :   1,979
   ChargePoint                             :   1,275
   flo                                     :     724
   PowerGo                                 :     421
   Punkt-E                                 :     366
   ChargeSini                              :     363
   Red E Charge                            :     278
   Blink Charging                          :     245
   (Business Owner at Location)            :  

## 8. Save Merged Dataset

In [24]:
print("💾 Saving merged dataset...")

# Create output directory if it doesn't exist
os.makedirs('../data/processed/merged_charging_station', exist_ok=True)

# Save to CSV
output_path = '../data/processed/merged_charging_station/ev_stations_merged_global.csv'
df_merged.to_csv(output_path, index=False)

file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
print(f"✓ Saved to: {output_path}")
print(f"✓ File size: {file_size:.2f} MB")
print(f"✓ Total records: {len(df_merged):,}")

print("\n" + "="*80)
print("📊 FINAL SUMMARY")
print("="*80)
print(f"\n📈 Dataset Statistics:")
print(f"   • Total charging stations: {len(df_merged):,}")
print(f"   • Countries covered: {df_merged['country'].nunique()}")
print(f"   • Unique operators: {df_merged['operator'].nunique()}")
print(f"   • International stations: {len(df_international):,} ({len(df_international)/len(df_merged)*100:.1f}%)")
print(f"   • Chinese stations: {len(df_chinese_aligned):,} ({len(df_chinese_aligned)/len(df_merged)*100:.1f}%)")

print(f"\n✅ SUCCESS! Global EV charging station database created!")
print("="*80)

💾 Saving merged dataset...


✓ Saved to: ../data/processed/merged_charging_station/ev_stations_merged_global.csv
✓ File size: 2.16 MB
✓ Total records: 12,002

📊 FINAL SUMMARY

📈 Dataset Statistics:
   • Total charging stations: 12,002
   • Countries covered: 72
   • Unique operators: 264
   • International stations: 10,000 (83.3%)
   • Chinese stations: 2,002 (16.7%)

✅ SUCCESS! Global EV charging station database created!

✓ File size: 2.16 MB
✓ Total records: 12,002

📊 FINAL SUMMARY

📈 Dataset Statistics:
   • Total charging stations: 12,002
   • Countries covered: 72
   • Unique operators: 264
   • International stations: 10,000 (83.3%)
   • Chinese stations: 2,002 (16.7%)

✅ SUCCESS! Global EV charging station database created!
